In [245]:
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io.fits import getdata
from astropy.time import Time
from astropy.io import fits

In [246]:
df = pd.read_csv('csv/full_merged.csv')
df.head()

,SL Target,SEQID,Module,Primary Target,Exposure (s),RA,DEC,PA,Unnamed: 8,In old sheet?,Notes
0,1RXSJ180408.9-342058,90102003002,A,SAX_J1808d4m3658,49267.8,272.1324,-36.9541,159.4761,1,Y,"Strong SL, unknown?"
1,1RXSJ180408.9-342058,90102003004,A,SAX_J1808d4m3658,21508.8,272.1424,-36.951,168.1145,1,Y,"Strong, unident SL"
2,1RXSJ180408.9-342058,90102003004,B,SAX_J1808d4m3658,21508.8,272.1424,-36.951,168.1145,1,Y,"Strong, unident SL"
3,2MASX J09172716-6456271,60401020002,A,IRAS_09149m6206,110899.0,138.9791,-62.3074,50.7186,1,N,SL on both
4,2MASX J09172716-6456271,60401020002,B,IRAS_09149m6206,110899.0,138.9791,-62.3074,50.7186,0,N,SL in both


In [247]:
greenlist = ['SL Target', 'SEQID', 'Module', 'Primary Target', 'Exposure (s)', 'RA',
       'DEC', 'Notes']
for col in df.columns:
    if col not in greenlist:
        df = df.drop(axis=1, labels=col)
        
# Drop everything with NaN in the SL Target column
df = df.dropna(subset=['SL Target'])
df = df.rename(columns={"Exposure (s)": "Exposure"})

df.head()

,SL Target,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes
0,1RXSJ180408.9-342058,90102003002,A,SAX_J1808d4m3658,49267.8,272.1324,-36.9541,"Strong SL, unknown?"
1,1RXSJ180408.9-342058,90102003004,A,SAX_J1808d4m3658,21508.8,272.1424,-36.951,"Strong, unident SL"
2,1RXSJ180408.9-342058,90102003004,B,SAX_J1808d4m3658,21508.8,272.1424,-36.951,"Strong, unident SL"
3,2MASX J09172716-6456271,60401020002,A,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL on both
4,2MASX J09172716-6456271,60401020002,B,IRAS_09149m6206,110899.0,138.9791,-62.3074,SL in both


In [248]:
# Set the "public" cutoff date to be June 1, 2020
cutoff_t = Time('2020-06-01T00:00:00.123456789', format='isot', scale='utc')
cutoff_mjd = cutoff_t.mjd

numaster = Table.read('fits/numaster.fits', format = 'fits')
    
dn = numaster.to_pandas()

greenlist = ['NAME', 'RA', 'DEC', 'TIME', 'END_TIME',
       'OBSID', 'EXPOSURE_A', 'EXPOSURE_B', 'PUBLIC_DATE']
for col in dn.columns:
    if col not in greenlist:
        dn.drop(col, axis=1, inplace=True)

dn['OBSID'] = pd.to_numeric(dn['OBSID'])
dn.columns

dn['PUBLIC'] = np.where((dn['PUBLIC_DATE']<cutoff_mjd)&(dn['PUBLIC_DATE'] > 0), 'yes', 'no')
dn.head()

,NAME,RA,DEC,TIME,END_TIME,OBSID,EXPOSURE_A,EXPOSURE_B,PUBLIC_DATE,PUBLIC
0,b'3C273_pcs11 ...,187.2386,2.0479,56109.875778,56109.941750,15011001,2881.8646,2882.2901,56533,yes
1,b'3C273_pcs12 ...,187.2038,2.0472,56109.941750,56110.011194,15012001,2880.8013,2894.2422,56533,yes
2,b'3C273_pcs13 ...,187.2367,2.0157,56110.011194,56110.080639,15013001,2572.5306,2575.3012,56533,yes
3,b'3C273_pcs14 ...,187.3142,2.0520,56110.080639,56110.146611,15014001,2607.3166,2609.0228,56533,yes
4,b'3C273_pcs15 ...,187.2341,2.1272,56110.146611,56110.212583,15015001,2844.8164,2847.9784,56533,yes


In [249]:

df['TIME'] = [-1. for x in range(len(df['SEQID']))]
df['END_TIME'] = [-1. for x in range(len(df['SEQID']))]

for si in np.unique(df['SEQID']):
#    print(dn.loc[dn['OBSID'] == si, 'TIME'].values[0])

    for mod in ['A', 'B']:
        
        df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'TIME'] = dn.loc[dn['OBSID'] == si, 'TIME'].values[0]
        df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'END_TIME'] = dn.loc[dn['OBSID'] == si, 'END_TIME'].values[0]
        df.loc[(df['SEQID'] == si)&(df['Module'] == mod), 'Exposure'] = dn.loc[dn['OBSID'] == si, f'EXPOSURE_{mod}'].values[0]
        

df.head()

,SL Target,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME
0,1RXSJ180408.9-342058,90102003002,A,SAX_J1808d4m3658,49264.2160,272.1324,-36.9541,"Strong SL, unknown?",57127.042444,57128.309805
1,1RXSJ180408.9-342058,90102003004,A,SAX_J1808d4m3658,21507.4594,272.1424,-36.951,"Strong, unident SL",57151.969528,57152.427861
2,1RXSJ180408.9-342058,90102003004,B,SAX_J1808d4m3658,21464.5109,272.1424,-36.951,"Strong, unident SL",57151.969528,57152.427861
3,2MASX J09172716-6456271,60401020002,A,IRAS_09149m6206,110898.9483,138.9791,-62.3074,SL on both,58323.306356,58325.788995
4,2MASX J09172716-6456271,60401020002,B,IRAS_09149m6206,110459.0713,138.9791,-62.3074,SL in both,58323.306356,58325.788995


In [250]:
obj_type = pd.read_csv('csv/target_info.csv')
green_list = ['SL Target', 'Type']
for col in obj_type.columns:
    if col not in green_list:
        if col in obj_type.columns:
            obj_type = obj_type.drop(axis=1,labels=col)
obj_type.head()

,SL Target,Type
0,1RXSJ180408.9-342058,LMXB-NS
1,2MASX J09172716-6456271,AGN
2,3C 273,AGN
3,4U 0614+091,LMXB-NS
4,4U 1323-619,LMXB-NS


In [251]:
# Add target classification and target type

df2 = df.copy()
nrows = len(df2['SEQID'])
print(nrows)

classification= ['SL' for x in range(nrows) ]
targ_type= ['??' for x in range(nrows) ]


df2.insert(loc=0, column='Classification', value=classification)
df2.insert(loc=2, column='Target Type', value=targ_type)

for target, ttype in zip(obj_type['SL Target'], obj_type['Type']):
    df2.loc[df2['SL Target'] == target, 'Target Type'] = ttype

    
for id in ['Unkn', 'Complex', 'Faint', 'GR']:
    df2.loc[df2['SL Target'] == id, 'Classification'] = id
    df2.loc[df2['SL Target'] == id, 'SL Target'] = '??'



df2

756


,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME
0,SL,1RXSJ180408.9-342058,LMXB-NS,90102003002,A,SAX_J1808d4m3658,49264.2160,272.1324,-36.9541,"Strong SL, unknown?",57127.042444,57128.309805
1,SL,1RXSJ180408.9-342058,LMXB-NS,90102003004,A,SAX_J1808d4m3658,21507.4594,272.1424,-36.951,"Strong, unident SL",57151.969528,57152.427861
2,SL,1RXSJ180408.9-342058,LMXB-NS,90102003004,B,SAX_J1808d4m3658,21464.5109,272.1424,-36.951,"Strong, unident SL",57151.969528,57152.427861
3,SL,2MASX J09172716-6456271,AGN,60401020002,A,IRAS_09149m6206,110898.9483,138.9791,-62.3074,SL on both,58323.306356,58325.788995
4,SL,2MASX J09172716-6456271,AGN,60401020002,B,IRAS_09149m6206,110459.0713,138.9791,-62.3074,SL in both,58323.306356,58325.788995
...,...,...,...,...,...,...,...,...,...,...,...,...
751,Unkn,??,??,90001008002,A,GK_Per,42347.7044,52.7837,43.8698,Unknown edge of FPMA,-1.000000,-1.000000
752,SL,XTE J1855-026,HMXB-NS,30360001002,A,Swift_J1843d5m0343,75612.3463,280.9001,-3.6847,"Weak SL, second source present",57887.861912,57889.552884
753,SL,XTE J1855-026,HMXB-NS,30360001002,B,Swift_J1843d5m0343,75324.7767,280.9001,-3.6847,Weak edge SL in both,57887.861912,57889.552884
754,SL,XTE J1855-026,HMXB-NS,40301004002,B,KES_75,95084.6915,281.5856,-3.0002,Weak SL on both?,58013.788995,58016.202190


In [252]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u


df2['SIMBAD_ID'] = 'NA'
df2['RA_SL'] = -999
df2['DEC_SL'] = -999

targets = df2['SL Target'].unique()


for target in targets:
    if 'Crab' in target:
        starget = 'Crab'
    elif '1145.1' in target:
        starget = '1E 1145.1-6141'
    elif target == '??':
        continue
    elif '1716' in target:
        starget = 'GRS 1716-249'
    elif '16320' in target:
        starget = 'IGR J16320-4751'
    elif 'M1812' in target:
        starget = '4U 1812-12'
    elif '1348' in target:
        starget = 'MAXI J1348-630'
    elif 'J1535' in target:
        starget = 'MAXI J1535-571'
    elif 'J1820' in target:
        starget = 'MAXI J1820+070'
    elif 'Coma' in target:
        starget = 'Coma Cluster'
    elif 'MAXI J1621' in target:
        starget = 'MAXI J1621-501'
    elif 'X1908' in target:
        starget = '1E 1908.4+0730'
    else:
        starget = target
        
    result_table = Simbad.query_object(starget)


    ra_tab = result_table['RA'].data.data[0]
    dec_tab = result_table['DEC'].data.data[0]
    coord = SkyCoord(ra_tab, dec_tab, unit=(u.hourangle, u.deg))
    
    
    df2.loc[df2['SL Target'] == target, 'SIMBAD_ID'] = starget
    df2.loc[df2['SL Target'] == target, 'RA_SL'] = coord.ra.deg
    df2.loc[df2['SL Target'] == target, 'DEC_SL'] = coord.dec.deg



#    print(f'{target} {starget} {coord.ra.deg} {coord.dec.deg}')
    #     ra = np.append(ra, coord.ra.deg)
#     dec = np.append(dec, coord.dec.deg)
#     gal_lat = np.append(gal_lat, coord.galactic.b.deg)
#     gal_lon = np.append(gal_lon, coord.galactic.l.deg)
#     names = np.append(names, target)

In [253]:
df2.sort_values(by=['RA', 'TIME', 'Module'], axis=0, inplace=True, ignore_index=True)

df2.head()

,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME,SIMBAD_ID,RA_SL,DEC_SL
0,Faint,??,??,30001014002,B,IC10_X1,88472.3252,5.0742,59.2736,"Maybe faint, SL?",56967.768139,56969.709111,NA,-999.000000,-999.000000
1,Unkn,??,??,90101010002,A,IGR_J00291p5934,38102.7867,7.2752,59.5982,NaN,57231.532039,57232.323706,NA,-999.000000,-999.000000
2,SL,SMC X-1,HMXB-NS,90201030002,A,SWIFT_J003233d6m7306,54920.5708,8.1969,-73.0956,Corner SL,57586.716067,57587.608428,SMC X-1,19.271436,-73.443337
3,SL,SMC X-1,HMXB-NS,90201041002,A,SMC_X3,38601.5531,13.0299,-72.4574,SL and second point source,57704.740373,57705.608428,SMC X-1,19.271436,-73.443337
4,SL,SMC X-1,HMXB-NS,30361002002,A,SXP_15d3,70650.3252,13.1118,-73.3731,Strong Unidentified SL (LMC?),58087.025106,58088.636217,SMC X-1,19.271436,-73.443337


In [254]:
slid = [f'StrayCatsI_{x}' for x in df2.index]
df2.insert(loc = 0, 
          column = 'STRAYID', 
          value = slid)

In [256]:
df2.head()

,STRAYID,Classification,SL Target,Target Type,SEQID,Module,Primary Target,Exposure,RA,DEC,Notes,TIME,END_TIME,SIMBAD_ID,RA_SL,DEC_SL
0,StrayCatsI_0,Faint,??,??,30001014002,B,IC10_X1,88472.3252,5.0742,59.2736,"Maybe faint, SL?",56967.768139,56969.709111,NA,-999.000000,-999.000000
1,StrayCatsI_1,Unkn,??,??,90101010002,A,IGR_J00291p5934,38102.7867,7.2752,59.5982,NaN,57231.532039,57232.323706,NA,-999.000000,-999.000000
2,StrayCatsI_2,SL,SMC X-1,HMXB-NS,90201030002,A,SWIFT_J003233d6m7306,54920.5708,8.1969,-73.0956,Corner SL,57586.716067,57587.608428,SMC X-1,19.271436,-73.443337
3,StrayCatsI_3,SL,SMC X-1,HMXB-NS,90201041002,A,SMC_X3,38601.5531,13.0299,-72.4574,SL and second point source,57704.740373,57705.608428,SMC X-1,19.271436,-73.443337
4,StrayCatsI_4,SL,SMC X-1,HMXB-NS,30361002002,A,SXP_15d3,70650.3252,13.1118,-73.3731,Strong Unidentified SL (LMC?),58087.025106,58088.636217,SMC X-1,19.271436,-73.443337


In [257]:
df2 = df2.rename(columns={"SL Target": "SL Source"})
df2 = df2.rename(columns={"Primary Target": "Primary"})
df2 = df2.rename(columns={"RA": "RA_PRIMARY"})
df2 = df2.rename(columns={"DEC": "DEC_PRIMARY"})
df2 = df2.rename(columns={"Target Type": "SL Type"})
df2

,STRAYID,Classification,SL Source,SL Type,SEQID,Module,Primary,Exposure,RA_PRIMARY,DEC_PRIMARY,Notes,TIME,END_TIME,SIMBAD_ID,RA_SL,DEC_SL
0,StrayCatsI_0,Faint,??,??,30001014002,B,IC10_X1,88472.3252,5.0742,59.2736,"Maybe faint, SL?",56967.768139,56969.709111,NA,-999.000000,-999.000000
1,StrayCatsI_1,Unkn,??,??,90101010002,A,IGR_J00291p5934,38102.7867,7.2752,59.5982,NaN,57231.532039,57232.323706,NA,-999.000000,-999.000000
2,StrayCatsI_2,SL,SMC X-1,HMXB-NS,90201030002,A,SWIFT_J003233d6m7306,54920.5708,8.1969,-73.0956,Corner SL,57586.716067,57587.608428,SMC X-1,19.271436,-73.443337
3,StrayCatsI_3,SL,SMC X-1,HMXB-NS,90201041002,A,SMC_X3,38601.5531,13.0299,-72.4574,SL and second point source,57704.740373,57705.608428,SMC X-1,19.271436,-73.443337
4,StrayCatsI_4,SL,SMC X-1,HMXB-NS,30361002002,A,SXP_15d3,70650.3252,13.1118,-73.3731,Strong Unidentified SL (LMC?),58087.025106,58088.636217,SMC X-1,19.271436,-73.443337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,StrayCatsI_751,SL,IGR J21247+5058,Radio galaxy,30460001002,B,1RXS_J21334d1p510725,26112.9105,323.4654,51.1078,Second source + faint SL on B,58172.535523,58173.056356,IGR J21247+5058,321.164167,50.973611
752,StrayCatsI_752,SL,Cas A,SNR,30001026002,A,1E2259p586,37295.5298,345.3277,58.8799,Unident stray light on FPMA,56406.910500,56407.719528,Cas A,350.850000,58.815000
753,StrayCatsI_753,SL,Cas A,SNR,30001026005,A,1E2259p586,16305.6674,345.3552,58.8882,Unident stray light on FPMA,56408.924389,56409.233416,Cas A,350.850000,58.815000
754,StrayCatsI_754,SL,Cas A,SNR,30001026007,A,1E2259p586,88381.2177,345.3570,58.8956,Unident stray light on FPMA,56428.396611,56430.250778,Cas A,350.850000,58.815000


In [258]:
exp = df2['Exposure']
df2.drop(labels=['Exposure'], axis=1,inplace = True)
df2.insert(6, 'Exposure', exp)

sl = df2['SL Source']
df2.drop(labels=['SL Source'], axis=1,inplace = True)
df2.insert(12, 'SL Source', sl)


sltype = df2['SL Type']
df2.drop(labels=['SL Type'], axis=1,inplace = True)
df2.insert(12, 'SL Type', sltype)


df2.head()

,STRAYID,Classification,SEQID,Module,Exposure,Primary,RA_PRIMARY,DEC_PRIMARY,Notes,TIME,END_TIME,SL Source,SL Type,SIMBAD_ID,RA_SL,DEC_SL
0,StrayCatsI_0,Faint,30001014002,B,88472.3252,IC10_X1,5.0742,59.2736,"Maybe faint, SL?",56967.768139,56969.709111,??,??,NA,-999.000000,-999.000000
1,StrayCatsI_1,Unkn,90101010002,A,38102.7867,IGR_J00291p5934,7.2752,59.5982,NaN,57231.532039,57232.323706,??,??,NA,-999.000000,-999.000000
2,StrayCatsI_2,SL,90201030002,A,54920.5708,SWIFT_J003233d6m7306,8.1969,-73.0956,Corner SL,57586.716067,57587.608428,SMC X-1,HMXB-NS,SMC X-1,19.271436,-73.443337
3,StrayCatsI_3,SL,90201041002,A,38601.5531,SMC_X3,13.0299,-72.4574,SL and second point source,57704.740373,57705.608428,SMC X-1,HMXB-NS,SMC X-1,19.271436,-73.443337
4,StrayCatsI_4,SL,30361002002,A,70650.3252,SXP_15d3,13.1118,-73.3731,Strong Unidentified SL (LMC?),58087.025106,58088.636217,SMC X-1,HMXB-NS,SMC X-1,19.271436,-73.443337
